In [1]:
pip install opencv-python opencv-python-headless numpy

Note: you may need to restart the kernel to use updated packages.


In [3]:
import cv2
import csv
import numpy as np

# Load the model
net = cv2.dnn.readNetFromCaffe("C://Users//admin//Downloads//deploy.prototxt", "C://Users//admin//Downloads//mobilenet_iter_73000.caffemodel")

# Define the list of class labels MobileNet SSD was trained to detect
CLASSES = ["background", "aeroplane", "bicycle", "bird", "boat",
           "bottle", "bus", "car", "cat", "chair", "cow", "diningtable",
           "dog", "horse", "motorbike", "person", "pottedplant", "sheep",
           "sofa", "train", "tvmonitor"]

# Load the video
cap = cv2.VideoCapture("C:/Users/admin/Downloads/Dog.mp4")

# Initialize variables
entry_times = {"person": None, "dog": None}
frame_count = 0
fps = cap.get(cv2.CAP_PROP_FPS)

# Function to perform non-maximum suppression
def non_max_suppression_fast(boxes, overlap_thresh):
    if len(boxes) == 0:
        return []

    if boxes.dtype.kind == "i":
        boxes = boxes.astype("float")

    pick = []

    x1 = boxes[:, 0]
    y1 = boxes[:, 1]
    x2 = boxes[:, 2]
    y2 = boxes[:, 3]

    area = (x2 - x1 + 1) * (y2 - y1 + 1)
    idxs = np.argsort(y2)

    while len(idxs) > 0:
        last = len(idxs) - 1
        i = idxs[last]
        pick.append(i)

        xx1 = np.maximum(x1[i], x1[idxs[:last]])
        yy1 = np.maximum(y1[i], y1[idxs[:last]])
        xx2 = np.minimum(x2[i], x2[idxs[:last]])
        yy2 = np.minimum(y2[i], y2[idxs[:last]])

        w = np.maximum(0, xx2 - xx1 + 1)
        h = np.maximum(0, yy2 - yy1 + 1)

        overlap = (w * h) / area[idxs[:last]]

        idxs = np.delete(idxs, np.concatenate(([last], np.where(overlap > overlap_thresh)[0])))

    return boxes[pick].astype("int")

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    
    frame_count += 1

    # Prepare the frame for object detection
    (h, w) = frame.shape[:2]
    blob = cv2.dnn.blobFromImage(cv2.resize(frame, (300, 300)), 0.007843, (300, 300), 127.5)
    net.setInput(blob)
    detections = net.forward()

    boxes = []
    labels = []
    for i in range(detections.shape[2]):
        confidence = detections[0, 0, i, 2]
        if confidence > 0.5:  # Adjusted confidence threshold
            idx = int(detections[0, 0, i, 1])
            label = CLASSES[idx]

            if label in ["person", "dog"]:
                if entry_times[label] is None:
                    entry_times[label] = frame_count / fps

                box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
                boxes.append(box)
                labels.append(label)

    if len(boxes) > 0:
        boxes = np.array(boxes)
        boxes = non_max_suppression_fast(boxes, 0.3)

        for i, (startX, startY, endX, endY) in enumerate(boxes):
            label = labels[i]
            # Draw the bounding box for the detected object
            cv2.rectangle(frame, (startX, startY), (endX, endY), (0, 255, 0), 2)
            cv2.putText(frame, label, (startX, startY - 15), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # Display the frame
    cv2.imshow("Frame", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

# Save the entry times to a CSV file
with open("object_entry_times.csv", "w", newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["Object Class", "Entry Time (seconds)"])
    for cls, entry_time in entry_times.items():
        if entry_time is not None:
            writer.writerow([cls, entry_time])

print("Entry times of detected objects:", entry_times)

 

Entry times of detected objects: {'person': 2.4, 'dog': 0.2}
